In [1]:
import pandas as pd
import swifter
import os

In [2]:
data_dir =  os.path.join('..','..','data')

In [3]:
data_dir =  os.path.join('..','..','data')
all_data_path = os.path.join(data_dir, 'allCountries.txt')

all_columns = ['geonameid', 'name', 'asciiname', 'alternate_names', 'latitude', 'longitude', 
               'feature class', 'feature code', 'country_code', 'cc2', 'admin1 code', 'admin2 code', 
               'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date']

basic_cols = ['geonameid', 'name', 'asciiname', 'alternate_names', 'country_code', 'cc2']

all_df = pd.read_csv(all_data_path, header=None, sep='\t', names=all_columns)
all_df = all_df.where((pd.notnull(all_df)), None)
all_df = all_df[basic_cols]

c:\users\giorg\.virtualenvs\lgm-similaritylearning-xx7dgzm9\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def get_train_test_datasets(fname):
    
    path = os.path.join(data_dir, fname
                       )
    split_df = pd.read_csv(path)
    
    test_ids= set(split_df[split_df['dataset'] == 'test']['geoname_id'])
    
    all_train_df = all_df[~all_df['geonameid'].isin(test_ids)].copy()
    
    all_test_df = all_df[all_df['geonameid'].isin(test_ids)].copy()
    
    return all_train_df, all_test_df

In [5]:
filename = 'unbiased_preshuffled_dataset-string-similarity-global-ids.csv'

In [6]:
train, test = get_train_test_datasets(filename)

In [7]:
train.shape

(10775495, 6)

In [8]:
test.shape

(1171871, 6)

In [9]:
def prepare_dataset(dataset):

    data = dataset.copy()
    
    data.dropna(subset=['alternate_names'], inplace=True)
    data['alternate_names'] = data['alternate_names'].apply(lambda x: x.split(','))
    data['alternate_names'] = data.swifter.apply(
        lambda row: [name for name in row['alternate_names'] if name.lower() != row['name'].lower()], axis=1)
    data = data[data['alternate_names'].apply(len) > 0]
    data['alternate_names'] = data['alternate_names'].apply(lambda x: ','.join(x) )
    
    return data

In [10]:
clean_train = prepare_dataset(train)

In [11]:
clean_test = prepare_dataset(test)

In [12]:
clean_train.shape

(3209456, 6)

In [13]:
clean_test.shape

(1171825, 6)

In [14]:
train_fpath = '-'.join(filename.split('-')[:-1] + ['train', 'original.csv'])
train_fpath = os.path.join(data_dir, train_fpath)
train_fpath

'..\\..\\data\\unbiased_preshuffled_dataset-string-similarity-global-train-original.csv'

In [15]:
test_fpath = '-'.join(filename.split('-')[:-1] + ['test', 'original.csv'])
test_fpath = os.path.join(data_dir, test_fpath)
test_fpath

'..\\..\\data\\unbiased_preshuffled_dataset-string-similarity-global-test-original.csv'

In [16]:
clean_train.to_csv(train_fpath, encoding='utf-8', index=False)

In [17]:
clean_test.to_csv(test_fpath, encoding='utf-8', index=False)

In [18]:
'Done'

'Done'

In [19]:
clean_train

,geonameid,name,asciiname,alternate_names,country_code,cc2
0,2986043,Pic de Font Blanca,Pic de Font Blanca,Pic du Port,AD,None
1,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler",AD,"AD,FR"
5,3023203,Port Vieux de la Coume d’Ose,Port Vieux de la Coume d'Ose,"Port Vieux de Coume d'Ose,Port Vieux de Coume ...",AD,None
6,3029315,Port de la Cabanette,Port de la Cabanette,Porteille de la Cabanette,AD,"AD,FR"
7,3034945,Port Dret,Port Dret,"Port de Bareites,Port de las Bareytes,Port des...",AD,None
20,3038827,Visanseny,Visanseny,Visanceny,AD,None
25,3038832,Vila,Vila,Casas Vila,AD,None
38,3038845,Port Negre del Pallars,Port Negre del Pallars,Port Vell,AD,ES
65,3038872,Riu Valira d’Orient,Riu Valira d'Orient,"Rio Balira del Orien,Riu Valira d'Orient,Río B...",AD,None
79,3038886,Pic de Tristaina,Pic de Tristaina,"Pic de Triatagne,Pic de Tristagne,Pic des Tri-...",AD,"AD,FR"
